<a href="https://colab.research.google.com/github/fatemafaria142/Improved-Language-Model-Instructions-Tuning-using-Alpaca-Dataset/blob/main/Instructions_Tuning_using_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [4]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.2 MB/s eta 0:00:00


In [5]:
!pip install accelerate -U

# **Dataset Link:** https://huggingface.co/datasets/tatsu-lab/alpaca?row=0

In [6]:
from datasets import load_dataset

dataset = load_dataset("tatsu-lab/alpaca")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Get the first 5000 data points**

In [9]:
# Get the first 5000 data points
num_samples_to_display = 5000
subset_dataset = dataset['train'].select(range(num_samples_to_display))

# Display information for 3 data points from the subset
num_samples_to_show = 3
for i in range(num_samples_to_show):
    data = subset_dataset[i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("Text:", data['text'])
    print("\n-----------------------------\n")

Data Point 1:
Instruction: Give three tips for staying healthy.
Input: 
Output: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.
Text: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.

-----------------------------

Data Point 2:
Instruction: What are the three primary colors?
Input: 
Output: The three primary colors are red, blue, and yellow.
Text: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are the three primary colors?

### 

# **Functions to Generate Prompts from Dataset Entries**

In [10]:
def get_prompt_with_input(x):
    result = f"### Instruction:\n{x['instruction']}\n\n"
    result += f"### Input:\n{x['input']}\n\n"
    result += f"### Response:\n{x['output']}"
    return result

def get_prompt_without_input(x):
    result = f"### Instruction:\n{x['instruction']}\n\n"
    result += f"### Response:\n{x['output']}"
    return result

def get_prompt(x):
    if x['input'] == '':
        return get_prompt_without_input(x)
    else:
        return get_prompt_with_input(x)


# **Display prompts for the first 5 data points**

In [11]:
# Generate prompts for each data point in the subset dataset
prompts = []
for i in range(num_samples_to_display):
    data = subset_dataset[i]
    prompt = get_prompt(data)
    prompts.append(prompt)

# Display the generated prompts or use them as needed
for idx, prompt in enumerate(prompts[:5]):  # Display prompts for the first 5 data points
    print(f"Prompt for Data Point {idx + 1}:")
    print(prompt)
    print("\n-----------------------------\n")


Prompt for Data Point 1:
### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.

-----------------------------

Prompt for Data Point 2:
### Instruction:
What are the three primary colors?

### Response:
The three primary colors are red, blue, and yellow.

-----------------------------

Prompt for Data Point 3:
### Instruction:
Describe the structure of an atom.

### Response:
An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.

-----------------------------

Prompt for Data Point 4:


# **More Examples of Prompts**

In [25]:
# Generate prompts for each data point in the subset dataset
prompts = []
for i in range(num_samples_to_display):
    data = subset_dataset[i]
    prompt = get_prompt(data)
    prompts.append(prompt)

# Display the generated prompts or use them as needed
for idx, prompt in enumerate(prompts[5:10]):  # Display prompts for the first 3 data points
    print(f"Prompt for Data Point {idx + 1}:")
    print(prompt)
    print("\n-----------------------------\n")


Prompt for Data Point 1:
### Instruction:
Identify the odd one out.

### Input:
Twitter, Instagram, Telegram

### Response:
Telegram

-----------------------------

Prompt for Data Point 2:
### Instruction:
Explain why the following fraction is equivalent to 1/4

### Input:
4/16

### Response:
The fraction 4/16 is equivalent to 1/4 because both numerators and denominators are divisible by 4. Dividing both the top and bottom numbers by 4 yields the fraction 1/4.

-----------------------------

Prompt for Data Point 3:
### Instruction:
Write a short story in third person narration about a protagonist who has to make an important career decision.

### Response:
John was at a crossroads in his life. He had just graduated college and was now facing the big decision of what career to pursue. After much deliberation, he decided that he wanted to be an accountant and help the financially disadvantaged. He had always been good with numbers and enjoyed seeing the tangible results of his work. 



# **GPT2 and its tokenizer**
* https://huggingface.co/docs/transformers/model_doc/gpt2

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# **Dataset Preparation and Tokenization for GPT-2 Training**
* This code segment appears to perform the following tasks:

* Assumes a dataset with specific keys ('instruction', 'input', 'output', and 'text').
* Iterates through the dataset to generate prompts based on the available data points.
* Tokenizes the generated prompts and output text using a tokenizer, preparing them as tensors.
* Compiles tokenized inputs, labels, and attention masks to be used for GPT-2 model training.

In [35]:
from transformers import Trainer, TrainingArguments
import torch
# Assuming subset_dataset is your dataset with 'instruction', 'input', 'output', and 'text' keys
dataset = subset_dataset  # No need for ['train'] if keys are 'instruction', 'input', 'output', and 'text'

# Initialize empty lists to store inputs, targets, and attention masks
input_ids = []
labels = []
attention_masks = []

# Tokenize and prepare the dataset
for data_point in dataset:
    # Generate prompt
    prompt = get_prompt(data_point)

    # Tokenize prompt
    tokenized_prompts = tokenizer(prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Tokenize output text (corrected text)
    tokenized_output = tokenizer(data_point['output'], return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Append tokenized inputs, labels, and attention masks
    input_ids.append(tokenized_prompts['input_ids'])
    labels.append(tokenized_output['input_ids'])
    attention_masks.append(tokenized_prompts['attention_mask'])

# Convert lists to tensors
input_ids = torch.stack(input_ids)
labels = torch.stack(labels)
attention_masks = torch.stack(attention_masks)

### **Printing input_ids, labels, and attention_masks for the first 5 examples**

In [34]:
# Assuming the code provided earlier to prepare the dataset is already executed

# Printing input_ids, labels, and attention_masks for the first 5 examples
for i in range(3):
    print(f"Example {i+1}:")
    print("Input IDs:", input_ids[i])
    print("Labels:", labels[i])
    print("Attention Mask:", attention_masks[i])
    print("-----------------------")


Example 1:
Input IDs: tensor([[21017, 46486,    25,   198, 23318,  1115,  9040,   329, 10589,  5448,
            13,   198,   198, 21017, 18261,    25,   198,    16,    13, 47659,
           257, 12974,  5496,   290,   787,  1654,   284,  2291,  6088,   286,
         15921,   290, 13701,    13,   220,   198,    17,    13, 32900,  7987,
           284,  1394,   534,  1767,  4075,   290,  1913,    13,   220,   198,
            18,    13,  3497,  1576,  3993,   290,  5529,   257,  6414,  3993,
          7269,    13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 

# **Dynamic Data Collation for GPT-2 Model Training**
* This code segment encompasses a class, GPT2DataCollator, which is designed to handle the collation of input features for GPT-2 model training. It dynamically checks the type of input features (whether they are dictionaries or tuples) and appropriately extracts input IDs, attention masks, and labels for padding and preparing the data to the same length. This data collation process is crucial for ensuring the uniformity and compatibility of the input features during the training of the GPT-2 model.

In [68]:
from transformers import Trainer, TrainingArguments
import torch


class GPT2DataCollator:
    def __call__(self, features):
        # Check if the features are dictionaries or tuples
        if isinstance(features[0], dict):
            input_ids = [feature['input_ids'] for feature in features]
            attention_masks = [feature['attention_mask'] for feature in features]
            labels = [feature['labels'] for feature in features]
        else:  # Assuming features are tuples
            input_ids = [feature[0] for feature in features]
            attention_masks = [feature[1] for feature in features]
            labels = [feature[2] for feature in features]

        # Pad inputs and labels to the same length
        input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
        attention_masks = torch.nn.utils.rnn.pad_sequence(attention_masks, batch_first=True, padding_value=0)
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=tokenizer.pad_token_id)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
            'labels': labels
        }


# **Define the Training Arguments and Trainer**

In [69]:
'''
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./gpt2-finetuned-version',    # Directory to save the model and checkpoints
    num_train_epochs=1,               # Number of training epochs
    per_device_train_batch_size=4,    # Batch size per device during training
    save_steps=1000,                  # Save checkpoint every X steps
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=500,                # Log training metrics every X steps
    evaluation_strategy="epoch",      # Evaluation strategy to adopt during training
    report_to="none",                 # Disable evaluation during training
    prediction_loss_only=True,        # Compute only the prediction loss
    warmup_steps=500# number of warmup steps for learning rate scheduler
    # Add any additional arguments as needed
)

# Initialize the Trainer with the custom data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    eval_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    data_collator=GPT2DataCollator()  # Use the custom data collator
)

# Start training
trainer.train()

trainer.save_model()
'''

'\n# Define the training arguments\ntraining_args = TrainingArguments(\n    output_dir=\'./gpt2-finetuned-version\',    # Directory to save the model and checkpoints\n    num_train_epochs=1,               # Number of training epochs\n    per_device_train_batch_size=4,    # Batch size per device during training\n    save_steps=1000,                  # Save checkpoint every X steps\n    logging_dir=\'./logs\',             # Directory for storing logs\n    logging_steps=500,                # Log training metrics every X steps\n    evaluation_strategy="epoch",      # Evaluation strategy to adopt during training\n    report_to="none",                 # Disable evaluation during training\n    prediction_loss_only=True,        # Compute only the prediction loss\n    warmup_steps=500# number of warmup steps for learning rate scheduler\n    # Add any additional arguments as needed\n)\n\n# Initialize the Trainer with the custom data collator\ntrainer = Trainer(\n    model=model,\n    args=traini

# **Load the save model and train it again here...**

In [76]:
from transformers import Trainer, TrainingArguments
# Load the previously trained model
model_path = './gpt2-finetuned-version'  # Replace this with the path to your saved model
model = AutoModelForCausalLM.from_pretrained(model_path)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./gpt2-final-version',    # Directory to save the model and checkpoints
    num_train_epochs=10,               # Number of training epochs
    per_device_train_batch_size=4,    # Batch size per device during training
    save_steps=1000,                  # Save checkpoint every X steps
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=500,                # Log training metrics every X steps
    evaluation_strategy="epoch",      # Evaluation strategy to adopt during training
    report_to="none",                 # Disable evaluation during training
    prediction_loss_only=True,        # Compute only the prediction loss
    warmup_steps=500# number of warmup steps for learning rate scheduler
    # Add any additional arguments as needed
)

# Initialize the Trainer with the custom data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    eval_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels),
    data_collator=GPT2DataCollator()  # Use the custom data collator
)

In [77]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.014100,1.947797
2,2.024200,1.853743
3,1.967700,1.773690
4,1.887900,1.686131
5,1.814000,1.604577
6,1.778200,1.538204
7,1.643700,1.478385
8,1.660500,1.433453
9,1.615900,1.408872
10,1.578300,1.396425


TrainOutput(global_step=12500, training_loss=1.7868629736328125, metrics={'train_runtime': 2951.6884, 'train_samples_per_second': 16.939, 'train_steps_per_second': 4.235, 'total_flos': 3266150400000000.0, 'train_loss': 1.7868629736328125, 'epoch': 10.0})

In [78]:
#Save the fine-tuned model here
trainer.save_model()

In [79]:
# Evaluate perplexity on the test dataset
eval_result = trainer.evaluate(eval_dataset=torch.utils.data.TensorDataset(input_ids, attention_masks, labels))
print("Perplexity:", eval_result['eval_loss'])

Perplexity: 1.3964250087738037


# **Text Generation Using Fine-Tuned GPT-2 Model Pipeline**

In [80]:
from transformers import pipeline

# Define the generator pipeline using the fine-tuned GPT-2 model and tokenizer
generator = pipeline('text-generation', model='./gpt2-final-version', tokenizer='gpt2')

# Test prompt for generating text
test_prompt = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\r\n\r\n"
    "### Instruction:\r\nList three technologies that make life easier.\r\n\r\n### Response:"
)

# Generate text based on the test prompt using the generator pipeline
generated_test_text = generator(test_prompt, max_length=100, num_return_sequences=1)

# Print the generated text for the test prompt
print(generated_test_text[0]['generated_text'])



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
List three technologies that make life easier.

### Response: reminders need reminders appointments need appointments access alarms money reminders reminders appointments internet


# **More Text Generation Examples**

In [84]:
from datasets import load_dataset

dataset = load_dataset("tatsu-lab/alpaca")

In [85]:
# Get the first 10 data points
num_samples_to_display = 10
subset_dataset = dataset['train'].select(range(num_samples_to_display))

# Display information for 3 data points from the subset
num_samples_to_show = 10
for i in range(num_samples_to_show):
    data = subset_dataset[i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("Text:", data['text'])
    print("\n-----------------------------\n")

Data Point 1:
Instruction: Give three tips for staying healthy.
Input: 
Output: 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.
Text: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response:
1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.

-----------------------------

Data Point 2:
Instruction: What are the three primary colors?
Input: 
Output: The three primary colors are red, blue, and yellow.
Text: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are the three primary colors?

### 

In [93]:
from transformers import pipeline

# Define the generator pipeline using the fine-tuned GPT-2 model and tokenizer
generator = pipeline('text-generation', model='./gpt2-final-version', tokenizer='gpt2', pad_token_id=tokenizer.pad_token_id)

# Generate text based on the subset of the training dataset
for example in subset_dataset:
    test_prompt = (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\r\n\r\n"
        f"### Instruction:\r\n{example['instruction']}\r\n\r\n### Response:"  # Replace this with your prompt logic
    )

    # Generate text based on the test prompt using the generator pipeline
    generated_text = generator(test_prompt, max_length=100, num_return_sequences=1)

    # Print the generated text for each test prompt
    print(generated_text[0]['generated_text'])
    print("------------------------------------------")


Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Give three tips for staying healthy.

### Response: enough mineralsrated and wash regularly,W
------------------------------------------
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What are the three primary colors?

### Response:
------------------------------------------
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Describe the structure of an atom.

### Response: components,rates atom. interaction electrons atom electrons the the electrons and electrons to, the groups atom. interactions:. electron bonds and electrons. the electrons atom with interact atom to form. atom the electrons. to bonds and used atom properties used to common used particles,
------------------------------------------
Below is an ins